# Bundling Analysis

### Load Libraries

In [15]:
# libraries
library(tidyverse)
library(arules)
library(arulesViz)
library(RPostgreSQL)
library(tsibble)
library(gridExtra)
library(clipr)
library(stringr)
library(config)
library(googlesheets4)

# setting wd
if ( grepl("recommender-system", getwd(), fixed = TRUE) ) {
    print("current working directory is fine")
} else {
    setwd("./projects/recommender-system")
    print("changed working directory")
}

[1] "current working directory is fine"


In [16]:
# load functions
source("bundle-functions.R")

### Get Data

In [17]:
# get credentials
dw <- config::get(file="./redshift_credentials.yml")

In [18]:
options(warn=-1)
# connecting to dw
con <- DBI::dbConnect(RPostgreSQL::PostgreSQL(),
                      dbname = dw$dbname,
                      host = dw$host,
                      port = dw$port,
                      user = dw$user,
                      password = dw$password)
# read query from sql file and execute
query_1 <- getSQL("sql/data_pull__purchases.sql") 
df_purchases <- dbGetQuery(con, query_1) %>% as_tibble()

query_2 <- getSQL("sql/data_pull__purchases_same_size_by_size.sql") 
df_purchases_same_size_by_size <- dbGetQuery(con, query_2) %>% as_tibble()

query_3 <- getSQL("sql/data_pull__purchases_same_size_overall.sql") 
df_purchases_same_size_overall <- dbGetQuery(con, query_3) %>% as_tibble()

query_4 <- getSQL("sql/data_pull__top-selling-designs.sql") 
df_top_designs <- dbGetQuery(con, query_4) %>% as_tibble()

options(warn=0)

In [21]:
head(df_purchases_same_size_overall)

customer_id,size,products_purchased
<dbl>,<chr>,<chr>
3394736324,8x10,"Kamran Hazel Rug, Watercolor Herringbone Blue Rug"
5228658244,3x5,"Camellia Jade Rug, Kamran Coral Rug, Starfish Pale Blue Rug"
5395331524,5x7,"Sudaria Solid Teal Blue Rug, Thalia Black Rug"
5564544452,2.5x7,"Leelu Black & White Rug, Prisma Natural Rug"
5568771844,5x7,"Maral Heriz Creme Rug, Moroccan Ornate Plush Rug"
340228636703,3x5,"Polar White Plush Rug, Watercolor Floral Coral Rug"


### Convert to Transactions

In [20]:
options(warn=-1)
purchase_transactions <- makeTransaction(df_purchases)
samesize_bysize_purchase_transactions <- makeTransaction(df_purchases_same_size_by_size)
samesize_overall_purchase_transactions <- makeTransaction(df_purchases_same_size_overall)
options(warn=0)

### Get Recommendations

In [24]:
rug_of_interest <- 'Domo Sage Rug'

In [26]:
select_rug_rules <- apriori(samesize_overall_purchase_transactions, parameter = list(supp=0.00001, conf=0.01),appearance = list(lhs=rug_of_interest,default="rhs"))
print(inspect(subset(select_rug_rules,lift > 1.2)))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.01    0.1    1 none FALSE            TRUE       5   1e-05      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 2 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[902 item(s), 226318 transaction(s)] done [0.08s].
sorting and recoding items ... [853 item(s)] done [0.00s].
creating transaction tree ... done [0.06s].
checking subsets of size 1 2 done [0.00s].
writing ... [72 rule(s)] done [0.00s].
creating S4 object  ... done [0.03s].
     lhs                rhs                                          support confidence    coverage      lift count
[1]  {Domo Sage Rug} => {Moden Blue Rug}                        3.092993e-05 0.01470588 0.002103235 13.366289     7
[2]  {Domo Sage Rug} => {Gradasi Sumac Rug}              